In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-01-18 17:05:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.73MB/s    in 0.5s    

2021-01-18 17:05:36 (1.73 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SportsETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
df.count()

4850360

In [ ]:
dropped_df = df.dropna()

In [ ]:
dropped_df.count()

4849000

In [ ]:
dropped_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
review_df = dropped_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
dup_review_df = review_df.dropDuplicates(["review_id"])

In [ ]:
dup_review_df.count()

4849000

In [ ]:
from pyspark.sql.types import * 

dup_review_df = dup_review_df.withColumn("customer_id",dup_review_df["customer_id"].cast(IntegerType()))
dup_review_df = dup_review_df.withColumn("product_parent",dup_review_df["product_parent"].cast(IntegerType()))
dup_review_df = dup_review_df.withColumn("review_date",dup_review_df["review_date"].cast(DateType()))

In [ ]:
dup_review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_table = dup_review_df
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10009GL6PAAAW|    4877998|B0059MN72W|     971531492| 2014-08-02|
|R1006JQWUSRBNN|   28109784|B005N0RX4S|     306278409| 2015-06-08|
|R1008DD0C9NX9Z|   20428086|B000NNLSMS|     956161586| 2010-07-22|
|R100AR6CZ42VWH|   42573673|B0045UYEVW|     491744919| 2015-03-29|
|R1010416UKW5ZU|   42049014|B000NNRHUK|     863410166| 2014-07-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
products = dropped_df.select(["product_id","product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B012P7UPSM|Chicago Blackhawk...|
|B001GQ3VHG|Copag Poker Size ...|
|B008VS8M58|Baoer 223 5.56x45...|
|B005F06F4U|All Terrain Tackl...|
|B010T4IE2C|Swim Cap - 3 Pack...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
customers = dropped_df.select(["customer_id"])
customers.show(5)

+-----------+
|customer_id|
+-----------+
|   48945260|
|    5782091|
|   45813853|
|    1593730|
|   29605511|
+-----------+
only showing top 5 rows



In [ ]:
customers = customers.groupBy("customer_id").count()
customers.orderBy("customer_id").select(["customer_id", "count"])
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
customers = customers.withColumn("count",customers["count"].cast(IntegerType()))
customers = customers.withColumnRenamed("count", "customer_count")
#customers.printSchema()
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45193257|             1|
|   44934924|             1|
|   11614909|             1|
|   19186664|             1|
|   50492011|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
vine_table = dropped_df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
vine_table = vine_table.withColumn("star_rating",vine_table["star_rating"].cast(IntegerType()))
vine_table = vine_table.withColumn("helpful_votes",vine_table["helpful_votes"].cast(IntegerType()))
vine_table = vine_table.withColumn("total_votes",vine_table["total_votes"].cast(IntegerType()))
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)